In [1]:
import pandas as pd
import datetime
df = pd.read_csv( "castle-event-data-v4.csv",sep=',', header=0, parse_dates=True,quoting=1 )


C:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
import numpy as np

In [3]:
#replace all values "custome" in column name with custom name
df.name=df.name.replace(to_replace='custom',value=np.nan)
df.name=df.name.fillna(df.custom_name)

In [4]:
df_model=df.drop(['ua','created_at','user_id','session_id','device_id','ip','lon','lat','accuracy','custom_name'],axis=1)
df_model=df_model.drop(['l','city','isp_name'],axis=1)

In [5]:
df_model.columns

Index([u'name', u'continent', u'country', u'datacenter', u'proxy',
       u'mobile_device'],
      dtype='object')

In [6]:
df.name.value_counts()

page_viewed                                   1535114
login_succeeded                                335810
passwordChange_succeeded                        73548
Payment With Balance                            62531
Regular Cellphone Confirmation                  26639
passwordReset_requested                         14829
Create Withdraw                                 13383
Account Create Edc Deposit On Landing Page      12417
registration_succeeded                          11175
Card Checkout                                    7152
Mobile Show Cvv                                  4975
Cellphone Confirmation                           3864
First Timeline Access                            3580
First Card Access                                3576
Mobile Withdraw Check                            1728
New Contact                                       463
Account Login Soft Block                          200
Mobile Account Change Password                      9
Regular Cellphone Confirmati

In [7]:
df_trans=pd.get_dummies(df_model,dummy_na=True)

In [8]:
df_trans.columns

Index([u'datacenter', u'proxy', u'mobile_device',
       u'name_Account Create Edc Deposit On Landing Page',
       u'name_Account Login Mobile Chat#using Token',
       u'name_Account Login Soft Block', u'name_Card Checkout',
       u'name_Cellphone Confirmation', u'name_Create Withdraw',
       u'name_First Card Access',
       ...
       u'country_Turkey', u'country_U.S. Virgin Islands', u'country_Ukraine',
       u'country_United Arab Emirates', u'country_United Kingdom',
       u'country_United States', u'country_Uruguay', u'country_Venezuela',
       u'country_Vietnam', u'country_nan'],
      dtype='object', length=123)

In [9]:
df_trans.shape

(2111004, 123)

In [10]:
# apparently we need to do 1 hot coding for the model
df_trans=pd.get_dummies(df_model,dummy_na=True)

In [11]:
df_trans.head()

,datacenter,proxy,mobile_device,name_Account Create Edc Deposit On Landing Page,name_Account Login Mobile Chat#using Token,name_Account Login Soft Block,name_Card Checkout,name_Cellphone Confirmation,name_Create Withdraw,name_First Card Access,...,country_Turkey,country_U.S. Virgin Islands,country_Ukraine,country_United Arab Emirates,country_United Kingdom,country_United States,country_Uruguay,country_Venezuela,country_Vietnam,country_nan
0,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,False,False,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#initialize isolation forest
from sklearn.ensemble import IsolationForest
seed=25
sf=IsolationForest(contamination=0.05,n_jobs=-1,random_state=seed)

In [ ]:
# fitting the isolation forest model
sf.fit(df_trans)


In [ ]:
df['score']=sf.predict(df_model)

In [ ]:
df[df.score==-1].device.value_counts()